# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UID peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple

La résultat final est dans la dernière cellule. Toutes les cellules précédentes sont là pour aider à la compréhension, en présentant des résultats intermédiaires.


In [1]:
#%load_ext sql
#%sql duckdb:///../../database/data.duckdb
#%config SqlMagic.displaylimit = 10

In [2]:
import duckdb
from pipelines.tasks.config.common import DUCKDB_FILE

import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

In [3]:
mesures_cat_communes = """
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),
    
mesures_cat  AS (
SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
        ),
        
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),
    
udi_cog AS (
  SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
 ),
    
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
)

    SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi_cog.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
"""

con.sql(mesures_cat_communes).show()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_8816\2727549379.py:1: SyntaxWarning: invalid escape sequence '\d'
  mesures_cat_communes = """


┌───────────────┬─────────────────────┬─────────────┬────────────┬──────────────────┬─────────┬───────────────────────────┬────────────┬────────────────────┐
│ referenceprel │ cdparametresiseeaux │ valtraduite │ limitequal │ limitequal_float │  unite  │         categorie         │  dateprel  │ commune_code_insee │
│    varchar    │       varchar       │   double    │  varchar   │      float       │ varchar │          varchar          │    date    │      varchar       │
├───────────────┼─────────────────────┼─────────────┼────────────┼──────────────────┼─────────┼───────────────────────────┼────────────┼────────────────────┤
│ 00200179905   │ TCEYTCL             │         0.0 │ <=10 µg/L  │             10.0 │ µg/L    │ hydrocarbure              │ 2020-05-19 │ 02781              │
│ 00200179905   │ TCLC                │         0.0 │ NULL       │             NULL │ NULL    │ non classé                │ 2020-05-19 │ 02781              │
│ 00200179905   │ TCLEY               │         0.0 

In [4]:
mesures_cat_communes_year = """
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),
    
mesures_cat  AS (
SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
        ),
        
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),
    
udi_cog AS (
  SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
 ),
    
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements),

mesures_cat_communes AS (
SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi_cog.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
)

SELECT 
 extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
 categorie,
 commune_code_insee,
 SUM(1) AS nb_analyses,
 SUM(case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
        else 0
    end ) AS nb_analyses_not_ok,
 SUM(case
        when limitequal_float is not NULL and valtraduite < limitequal_float then 1
        else 0
    end ) AS nb_analyses_ok,    
FROM
 mesures_cat_communes
GROUP BY 
    1,2,3
"""

con.sql(mesures_cat_communes_year).show()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_8816\1472271575.py:1: SyntaxWarning: invalid escape sequence '\d'
  mesures_cat_communes_year = """


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬───────────────────────────┬────────────────────┬─────────────┬────────────────────┬────────────────┐
│ annee │         categorie         │ commune_code_insee │ nb_analyses │ nb_analyses_not_ok │ nb_analyses_ok │
│ int64 │          varchar          │      varchar       │   int128    │       int128       │     int128     │
├───────┼───────────────────────────┼────────────────────┼─────────────┼────────────────────┼────────────────┤
│  2020 │ sous produit désinfection │ 04178              │         840 │                  0 │            320 │
│  2020 │ sous produit désinfection │ 04159              │         210 │                  0 │             90 │
│  2020 │ radioactivité             │ 04130              │          90 │                  0 │              0 │
│  2020 │ nitrite                   │ 04130              │         140 │                  0 │            140 │
│  2020 │ paramètre organoleptique  │ 04132              │         550 │                  0 │              5 │
│

In [7]:
mesures_cat_communes_year_cvm = """
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),
    
mesures_cat  AS (
SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
        ),
        
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),
    
udi_cog AS (
  SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
 ),
    
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements),

mesures_cat_communes AS (
SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi_cog.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
),

mesures_cat_communes_year AS (
SELECT 
 extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
 categorie,
 commune_code_insee,
 SUM(1) AS nb_analyses,
 SUM(case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
        else 0
    end ) AS nb_analyses_not_ok,
 SUM(case
        when limitequal_float is not NULL and valtraduite < limitequal_float then 1
        else 0
    end ) AS nb_analyses_ok,    
FROM
 mesures_cat_communes
GROUP BY 
    1,2,3
),

annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    )
    
SELECT
    annees.annee,
    commune_code_insee,
    CASE WHEN nb_analyses IS NULL OR nb_analyses = 0  THEN 'Pas recherché'
         WHEN nb_analyses_ok =0 THEN 'jamais quantifié'
         WHEN nb_analyses_ok >0 THEN '<= 0,5 µg/L'
         WHEN nb_analyses_not_ok >1 THEN '> 0,5 µg/L'
    END AS resultat
FROM
 annees -- pour avoir toutes les annees, même sans résultats
LEFT JOIN
 mesures_cat_communes_year
ON 
   annees.annee = mesures_cat_communes_year.annee
WHERE
 categorie = 'cvm'
"""

con.sql(mesures_cat_communes_year_cvm).show()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_8816\2414602722.py:1: SyntaxWarning: invalid escape sequence '\d'
  mesures_cat_communes_year_cvm = """


┌───────┬────────────────────┬─────────────┐
│ annee │ commune_code_insee │  resultat   │
│ int64 │      varchar       │   varchar   │
├───────┼────────────────────┼─────────────┤
│  2020 │ 83054              │ <= 0,5 µg/L │
│  2020 │ 83001              │ <= 0,5 µg/L │
│  2020 │ 83150              │ <= 0,5 µg/L │
│  2020 │ 83085              │ <= 0,5 µg/L │
│  2020 │ 83113              │ <= 0,5 µg/L │
│  2020 │ 83090              │ <= 0,5 µg/L │
│  2020 │ 83127              │ <= 0,5 µg/L │
│  2020 │ 83070              │ <= 0,5 µg/L │
│  2020 │ 84069              │ <= 0,5 µg/L │
│  2020 │ 84088              │ <= 0,5 µg/L │
│    ·  │   ·                │      ·      │
│    ·  │   ·                │      ·      │
│    ·  │   ·                │      ·      │
│  2023 │ 57180              │ <= 0,5 µg/L │
│  2023 │ 58259              │ <= 0,5 µg/L │
│  2023 │ 58159              │ <= 0,5 µg/L │
│  2023 │ 58199              │ <= 0,5 µg/L │
│  2023 │ 57073              │ <= 0,5 µg/L │
│  2023 │ 

# Check

In [8]:
Test = """
SELECT
    de_partition as year,
    COUNT(DISTINCT inseecommune)
FROM 
 edc_communes
GROUP BY 
    1
"""

con.sql(Test).show()

┌───────┬──────────────────────────────┐
│ year  │ count(DISTINCT inseecommune) │
│ int32 │            int64             │
├───────┼──────────────────────────────┤
│  2021 │                        34833 │
│  2020 │                        34788 │
│  2022 │                        34874 │
│  2024 │                        34809 │
│  2023 │                        34852 │
└───────┴──────────────────────────────┘



In [9]:
%%sql --with mesures_cat_communes_year_cvm  
 SELECT 
    annee,
    COUNT(DISTINCT commune_code_insee)
 FROM
   mesures_cat_communes_year_cvm
GROUP BY 
    1

UsageError: Cell magic `%%sql` not found.


In [10]:
finale = """
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),
    
mesures_cat  AS (
SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
        ),
        
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),
    
udi_cog AS (
  SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
 ),
    
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements),

mesures_cat_communes AS (
SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi_cog.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
),

mesures_cat_communes_year AS (
SELECT 
 extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
 categorie,
 commune_code_insee,
 SUM(1) AS nb_analyses,
 SUM(case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
        else 0
    end ) AS nb_analyses_not_ok,
 SUM(case
        when limitequal_float is not NULL and valtraduite < limitequal_float then 1
        else 0
    end ) AS nb_analyses_ok,    
FROM
 mesures_cat_communes
GROUP BY 
    1,2,3
),

annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),


mesures_cat_communes_year_cvm AS (
SELECT
    annees.annee,
    commune_code_insee,
    CASE WHEN nb_analyses IS NULL OR nb_analyses = 0 THEN 'Pas recherché'
         WHEN nb_analyses_ok =0 THEN 'jamais quantifié'
         WHEN nb_analyses_ok >0 THEN '<= 0,5 µg/L'
         WHEN nb_analyses_not_ok >1 THEN '> 0,5 µg/L'
    END AS resultat
FROM   
 annees  -- pour avoir toutes les années , même sans résultats
LEFT JOIN
 mesures_cat_communes_year
ON 
   annees.annee = mesures_cat_communes_year.annee
WHERE
 categorie = 'cvm'
 )

 SELECT 
    annee,
    COUNT(DISTINCT commune_code_insee)
 FROM
   mesures_cat_communes_year_cvm
GROUP BY 
    1
"""

con.sql(finale).show()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_8816\836711291.py:1: SyntaxWarning: invalid escape sequence '\d'
  finale = """


┌───────┬────────────────────────────────────┐
│ annee │ count(DISTINCT commune_code_insee) │
│ int64 │               int64                │
├───────┼────────────────────────────────────┤
│  2020 │                              32345 │
│  2021 │                              32456 │
│  2023 │                              32309 │
│  2022 │                              32619 │
│  2024 │                              32338 │
└───────┴────────────────────────────────────┘



**il manque des communes**

In [12]:
finale = """
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),
    
mesures_cat  AS (
SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
        ),
        
udi AS (
    SELECT
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),
    
udi_cog AS (
  SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
 ),
    
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements),

mesures_cat_communes AS (
SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi_cog.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
),

mesures_cat_communes_year AS (
SELECT 
 extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
 categorie,
 commune_code_insee,
 SUM(1) AS nb_analyses,
 SUM(case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
        else 0
    end ) AS nb_analyses_not_ok,
 SUM(case
        when limitequal_float is not NULL and valtraduite < limitequal_float then 1
        else 0
    end ) AS nb_analyses_ok,    
FROM
 mesures_cat_communes
GROUP BY 
    1,2,3
),

annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

mesures_cat_communes_year_cvm AS (
SELECT
    annees.annee,
    commune_code_insee,
    CASE WHEN nb_analyses IS NULL OR nb_analyses = 0 THEN 'Pas recherché'
         WHEN nb_analyses_ok =0 THEN 'jamais quantifié'
         WHEN nb_analyses_ok >0 THEN '<= 0,5 µg/L'
         WHEN nb_analyses_not_ok >1 THEN '> 0,5 µg/L'
    END AS resultat
FROM  
 annees  -- pour avoir toutes les années , même sans résultats
LEFT JOIN
 mesures_cat_communes_year
ON 
   annees.annee = mesures_cat_communes_year.annee
WHERE
 categorie = 'cvm'
 )

 SELECT 
    resultat,
    annee,
    COUNT(DISTINCT commune_code_insee)
 FROM
   mesures_cat_communes_year_cvm
GROUP BY 
    1,2
ORDER BY 2
"""

con.sql(finale).show()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Vinca\AppData\Local\Temp\ipykernel_8816\810035824.py:1: SyntaxWarning: invalid escape sequence '\d'
  finale = """


┌──────────────────┬───────┬────────────────────────────────────┐
│     resultat     │ annee │ count(DISTINCT commune_code_insee) │
│     varchar      │ int64 │               int64                │
├──────────────────┼───────┼────────────────────────────────────┤
│ <= 0,5 µg/L      │  2020 │                              32340 │
│ jamais quantifié │  2020 │                                  5 │
│ jamais quantifié │  2021 │                                 11 │
│ <= 0,5 µg/L      │  2021 │                              32445 │
│ jamais quantifié │  2022 │                                  5 │
│ <= 0,5 µg/L      │  2022 │                              32614 │
│ jamais quantifié │  2023 │                                  8 │
│ <= 0,5 µg/L      │  2023 │                              32301 │
│ jamais quantifié │  2024 │                                 11 │
│ <= 0,5 µg/L      │  2024 │                              32327 │
├──────────────────┴───────┴────────────────────────────────────┤
│ 10 rows 